Developed by: Thomas.Reimann@tu-dresden.de / Sophie.Pfoertner@mailbox.tu-dresden.de / Anne.Pfoertner@mailbox.tu-dresden.de / Alexander.Oy@mailbox.tu-dresden.de
<br>Last change: 2020 11 09
<br>Current state: funktional
<br>ToDo:
- Quellcode kommentieren
- Tabelle ordentlich machen --> Momentan noch mehrere tabellen (ersten zwei Zeilen und dann für jede Schleife ne neue Tabelle), diese von der Spaltenbreite gleich groß machen, formatierung der Nachkommastellen
- Eingangswerte etvl mit ipywidgets ins Modell
- Statt Frage ob neue Schleife, evtl Button einfügen

In [41]:
import matplotlib.pyplot as plt
import numpy as np
from numpy import nan as NaN
from ipywidgets import *
from IPython.display import display
import pandas as pd
from IPython.display import clear_output

In [46]:
button = Button(description="Berechnen")
output = Output()
out_plot = Output()

dx = 500                # Zellweite

m = 12                  # Mächtigkeit
imax = 100              # maximale Iterationszahl
epsilon = 0.00001       # Abbruchkriterium
ncol = 11             # Anzahl der Zellen
  
#Zonierung
Anteil_Zone1 = 0.4      # Anteil der Zone 1 
K1 = 2e-4               # hydraulische Leitfähigkeit der Zone 1
K2 = 2e-4               # hydraulische Leitfähigkeit der Zone 2
GWN1 = 0                # Grundwasserneubildung der Zone 1 in mm/a
GWN2 = 0                # Grundwasserneubildung der Zone 2 in mm/a

RCH1 = GWN1/1000/365.25/24/3600        #Umrechnung der Grundwasserneubildung in m/s
RCH2 = GWN2/1000/365.25/24/3600 

K_Zone = []
RCH_Zone = []
for z in range (0, int(ncol*Anteil_Zone1)):
    K_Zone.append(K1)
    RCH_Zone.append(RCH1)
for z in range (int(ncol*Anteil_Zone1),ncol):
    K_Zone.append(K2)
    RCH_Zone.append(RCH2)
print(int(ncol*Anteil_Zone1))
print(K_Zone)
print(RCH_Zone)

4
[0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [47]:
# Generate Dataframe and initiate the first row
col = []
leerZeile = []
for n in range (ncol):
    col.append('h'+ str(n+1))  # COLUMN HEADER, START WITH 1
    leerZeile.append(NaN)      # EMPTY ROW

global i
i = 0
global HEAD
HEAD = pd.DataFrame([leerZeile], columns=col)

print(HEAD)

   h1  h2  h3  h4  h5  h6  h7  h8  h9  h10  h11
0 NaN NaN NaN NaN NaN NaN NaN NaN NaN  NaN  NaN


In [48]:
#Randbedingungen/ Startwerte der Zellen
HEAD[col[ 0]].values[i] = 18               # Potentialhöhe Links in m
HEAD[col[-1]].values[i] = 16              # Potentialhöhe Rechts in m

# SET INITIAL VALUE TO THE AVERAGE OF BOTH BOUNDARIES
for n in range(1,ncol-1):
    HEAD[col[n]].values[i] = (HEAD[col[0]].values[i] + HEAD[col[-1]].values[i])/2

print(HEAD)

     h1    h2    h3    h4    h5    h6    h7    h8    h9   h10   h11
0  18.0  17.0  17.0  17.0  17.0  17.0  17.0  17.0  17.0  17.0  16.0


In [49]:
  
def iterator(HEAD=HEAD, RCH_Zone=RCH_Zone, K_Zone=K_Zone, dx=dx, i=i, col=col, m=m):
    T = []
    # FIRST ITERATION
    if i==0:
        i = i + 1
        NEW_LINE = pd.DataFrame([leerZeile], columns=col) 
        HEAD = HEAD.append(NEW_LINE, ignore_index=True)
        HEAD[col[ 0]].values[i] = HEAD[col[ 0]].values[i-1]
        HEAD[col[-1]].values[i] = HEAD[col[-1]].values[i-1]
        for n in range(0,ncol):
            T.append(K_Zone[n] * min([m, HEAD[col[n]].values[i-1]]))                     #Transmissivität
        for n in range(1,ncol-1):
            HEAD[col[n]].values[i]=(2*T[n]*T[n-1]/(T[n]+T[n-1])*HEAD[col[n-1]].values[i-1]+2*T[n]*T[n+1]/(T[n]+T[n+1])*HEAD[col[n+1]].values[i-1]+RCH_Zone[n]*dx**2)/(2*T[n]*T[n-1]/(T[n]+T[n-1])+2*T[n]*T[n+1]/(T[n]+T[n+1]))
        HEAD.style.format("{:.3f}") 
        with output:
            clear_output()
            print(HEAD)
        with out_plot:
            clear_output()
            HEAD[-1:].T.plot()
            plt.show()
        return HEAD, i    
    else:
        i = i + 1
        max_head_change = max(abs(HEAD.iloc[-2]-HEAD.iloc[-1]))
        if max_head_change<epsilon or i==imax:
            with output:
                print('Die Iteration wurde beendet.')
        else:
            NEW_LINE = pd.DataFrame([leerZeile], columns=col) 
            HEAD = HEAD.append(NEW_LINE, ignore_index=True)
            HEAD[col[ 0]].values[i] = HEAD[col[ 0]].values[i-1]
            HEAD[col[-1]].values[i] = HEAD[col[-1]].values[i-1]
            for n in range(0,ncol):
                T.append(K_Zone[n] * min([m, HEAD[col[n]].values[i-1]]))                     #Transmissivität
            for n in range(1,ncol-1):
                HEAD[col[n]].values[i]=(2*T[n]*T[n-1]/(T[n]+T[n-1])*HEAD[col[n-1]].values[i-1]+2*T[n]*T[n+1]/(T[n]+T[n+1])*HEAD[col[n+1]].values[i-1]+RCH_Zone[n]*dx**2)/(2*T[n]*T[n-1]/(T[n]+T[n-1])+2*T[n]*T[n+1]/(T[n]+T[n+1]))
            HEAD.style.format("{:.3f}") 
            with output:
                clear_output()
                print(HEAD)              
            with out_plot:
                clear_output()
                HEAD[-1:].T.plot()
                plt.show()
            return HEAD, i

def Iteration(_):
    global HEAD, i
    HEAD,i = iterator(HEAD=HEAD,i=i)

button.on_click(Iteration)
box = HBox([VBox([button, output]), out_plot])
box